# Setting up Environment

In [1]:
import math, os, re, time
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from tqdm.notebook import tqdm

tqdm.pandas()
%matplotlib inline

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

# Data Preparation

In [2]:
raw_data = pd.read_csv('../input/bt4222-project/modelling_dataset.csv')
raw_data = pd.concat([raw_data.pop('Unit Price ($ PSM)'), raw_data], axis=1)

raw_data

,Unit Price ($ PSM),Area (SQM),Floor Number,PPI,Average Cases Per Year,Nearest Primary School,nearest_station_distance,Remaining Lease,Ang Mo Kio,Bedok,Bishan,Bukit Batok,Bukit Merah,Bukit Panjang,Bukit Timah,Choa Chu Kang,Clementi,Downtown Core,Geylang,Hougang,Jurong East,Jurong West,Kallang,Mandai,Marine Parade,Museum,Newton,Novena,Orchard,Outram,Pasir Ris,Punggol,Queenstown,River Valley,Rochor,Sembawang,Sengkang,Serangoon,Singapore River,Southern Islands,Tampines,Tanglin,Toa Payoh,Yishun,BLUE,BROWN,GREEN,LRT,PURPLE,RED,YELLOW,Apartment,Executive Condominium
0,6316.0,95.0,1.0,124.3,33,705.752731,1207.822015,87.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,13833.0,138.0,1.0,124.3,58,1233.947139,768.529003,88.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,8990.0,99.0,10.0,124.3,50,1039.586179,816.818037,80.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,6306.0,111.0,6.0,124.3,33,509.516515,501.364218,80.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
4,13934.0,122.0,10.0,124.3,58,1253.733260,554.491114,88.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54669,8229.0,192.0,4.0,153.3,66,526.868822,464.021930,76.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
54670,7838.0,111.0,14.0,153.3,66,535.286185,490.669867,80.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1
54671,13028.0,142.0,14.0,153.3,133,2075.609094,600.145285,72.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
54672,13931.0,58.0,6.0,153.3,51,397.651025,539.116552,91.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


# Train-Test Split

In [3]:
# Perform 70/30 train_test split
X = raw_data.iloc[:,1:].copy()
y = raw_data.iloc[:,0:1].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3,
                                                    shuffle=True, 
                                                    random_state=42)
X_train

,Area (SQM),Floor Number,PPI,Average Cases Per Year,Nearest Primary School,nearest_station_distance,Remaining Lease,Ang Mo Kio,Bedok,Bishan,Bukit Batok,Bukit Merah,Bukit Panjang,Bukit Timah,Choa Chu Kang,Clementi,Downtown Core,Geylang,Hougang,Jurong East,Jurong West,Kallang,Mandai,Marine Parade,Museum,Newton,Novena,Orchard,Outram,Pasir Ris,Punggol,Queenstown,River Valley,Rochor,Sembawang,Sengkang,Serangoon,Singapore River,Southern Islands,Tampines,Tanglin,Toa Payoh,Yishun,BLUE,BROWN,GREEN,LRT,PURPLE,RED,YELLOW,Apartment,Executive Condominium
11187,129.0,9.0,139.2,66,313.319878,1514.605671,85.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
18593,66.0,13.0,142.3,63,408.305136,1517.388799,84.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27013,61.0,58.0,137.7,85,1529.338980,148.651892,86.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,1,0
39116,105.0,18.0,141.6,66,270.671774,225.750354,86.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
6042,111.0,37.0,132.6,60,339.216214,258.201238,87.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44732,107.0,3.0,145.2,36,102.333066,622.895879,73.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
54343,66.0,2.0,153.3,64,414.799433,805.354104,87.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
38158,74.0,3.0,141.6,142,774.750823,466.130105,76.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0
860,91.0,14.0,124.3,133,658.187930,310.779399,83.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0


# Feature Scaling

In [4]:
def all_transform(X_train, X_test):
    all_features = list(X_train.columns)

    standardScale_vars = ['Area (SQM)',
                          'Floor Number',
                          'PPI',
                          'Average Cases Per Year',
                          'Nearest Primary School',
                          'nearest_station_distance']

    minMax_vars = ['Remaining Lease']

    remaining_features = [x for x in all_features if x not in standardScale_vars and x not in minMax_vars]

    s_scaler = StandardScaler()
    mm_scaler = MinMaxScaler()

    s_scaled = pd.DataFrame(s_scaler.fit_transform(X_train.loc[:, standardScale_vars].copy()), columns=standardScale_vars, index=X_train.index)
    mm_scaled = pd.DataFrame(mm_scaler.fit_transform(X_train.loc[:, minMax_vars].copy()), columns=minMax_vars, index=X_train.index)

    X_train = pd.concat([s_scaled, 
                         mm_scaled, 
                         X_train.loc[:, remaining_features].copy()], axis=1)

    s_scaled = pd.DataFrame(s_scaler.transform(X_test.loc[:, standardScale_vars].copy()), columns=standardScale_vars, index=X_test.index)
    mm_scaled = pd.DataFrame(mm_scaler.transform(X_test.loc[:, minMax_vars].copy()), columns=minMax_vars, index=X_test.index)

    X_test = pd.concat([s_scaled, 
                        mm_scaled, 
                        X_test.loc[:, remaining_features].copy()], axis=1)
    
    return X_train, X_test

In [5]:
# Feature Scaling - fit_transform(training set), transform(test set)
X_train, X_test = all_transform(X_train, X_test)
X_train

,Area (SQM),Floor Number,PPI,Average Cases Per Year,Nearest Primary School,nearest_station_distance,Remaining Lease,Ang Mo Kio,Bedok,Bishan,Bukit Batok,Bukit Merah,Bukit Panjang,Bukit Timah,Choa Chu Kang,Clementi,Downtown Core,Geylang,Hougang,Jurong East,Jurong West,Kallang,Mandai,Marine Parade,Museum,Newton,Novena,Orchard,Outram,Pasir Ris,Punggol,Queenstown,River Valley,Rochor,Sembawang,Sengkang,Serangoon,Singapore River,Southern Islands,Tampines,Tanglin,Toa Payoh,Yishun,BLUE,BROWN,GREEN,LRT,PURPLE,RED,YELLOW,Apartment,Executive Condominium
11187,0.291530,-0.109176,-0.236229,0.243812,-0.771624,1.226898,0.745098,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
18593,-1.125873,0.371998,0.210453,0.125051,-0.570640,1.231468,0.725490,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
27013,-1.238366,5.785202,-0.452365,0.995966,1.801408,-1.015899,0.764706,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,1,0
39116,-0.248433,0.973465,0.109589,0.243812,-0.861865,-0.889309,0.764706,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0
6042,-0.113442,3.259040,-1.187229,0.006290,-0.716828,-0.836027,0.784314,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44732,-0.203436,-0.830937,0.628316,-0.943799,-1.218061,-0.237225,0.509804,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
54343,-1.125873,-0.951230,1.795452,0.164638,-0.556898,0.062359,0.784314,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
38158,-0.945886,-0.830937,0.109589,3.252428,0.204740,-0.494623,0.568627,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0
860,-0.563412,0.492291,-2.383183,2.896144,-0.041901,-0.749697,0.705882,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0


# Train-Validation Split

In [6]:
# Train-Eval split
X_train, X_eval, y_train, y_eval = train_test_split(X_train, y_train, 
                                                    test_size=0.2,
                                                    shuffle=True, 
                                                    random_state=42)
X_train

,Area (SQM),Floor Number,PPI,Average Cases Per Year,Nearest Primary School,nearest_station_distance,Remaining Lease,Ang Mo Kio,Bedok,Bishan,Bukit Batok,Bukit Merah,Bukit Panjang,Bukit Timah,Choa Chu Kang,Clementi,Downtown Core,Geylang,Hougang,Jurong East,Jurong West,Kallang,Mandai,Marine Parade,Museum,Newton,Novena,Orchard,Outram,Pasir Ris,Punggol,Queenstown,River Valley,Rochor,Sembawang,Sengkang,Serangoon,Singapore River,Southern Islands,Tampines,Tanglin,Toa Payoh,Yishun,BLUE,BROWN,GREEN,LRT,PURPLE,RED,YELLOW,Apartment,Executive Condominium
9079,-0.338427,-0.830937,-0.510002,-0.072884,-0.337170,-0.184168,0.843137,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
50500,0.426521,-0.349763,1.147043,2.896144,-0.084831,-0.944134,0.568627,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0
42324,-0.068445,1.334345,0.757998,-1.022973,-0.477199,-0.716844,0.882353,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0
33190,2.496380,0.612585,-1.028729,0.006290,0.510938,0.544301,0.862745,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12391,-0.023448,-0.590350,-0.005684,-0.864625,-0.737739,0.647023,0.803922,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40185,-0.225935,0.371998,0.757998,0.243812,-0.729890,0.154151,0.862745,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
46425,0.336527,0.612585,1.319952,0.243812,0.209585,-0.999626,0.725490,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
33159,-0.068445,-0.951230,-1.028729,-0.943799,-1.148569,-0.292310,0.549020,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
46563,-0.810895,0.492291,1.319952,-0.825038,0.254966,-0.725711,0.882353,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0


In [7]:
print(X_train.shape)
print(y_train.shape)
print(X_eval.shape)
print(y_eval.shape)
print(X_test.shape)
print(y_test.shape)

(30616, 52)
(30616, 1)
(7655, 52)
(7655, 1)
(16403, 52)
(16403, 1)


# Model Building

In [8]:
linear = LinearRegression()
lasso = LassoCV(random_state=42)
ridge = RidgeCV()
rfr = RandomForestRegressor(random_state=42)
xgb = XGBRegressor(random_state=42)
lgb = LGBMRegressor(random_state=42)

estimators = [
    ('linear', linear),
    ('lasso', lasso),
    ('ridge', ridge),
    ('random_forest', rfr),
    ('xgboost', xgb),
    ('lightgbm', lgb)
]

In [9]:
def best_meta_learner(estimators, meta_learner, meta_name):
    stacking = StackingRegressor(estimators=estimators,
                                 final_estimator=meta_learner,
                                 cv=5, 
                                 n_jobs=-1)
    stacking.fit(X_train, y_train)

    y_train_pred = stacking.predict(X_train)
    y_eval_pred = stacking.predict(X_eval)
    y_test_pred = stacking.predict(X_test)

    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    eval_rmse = np.sqrt(mean_squared_error(y_eval, y_eval_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

    train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
    eval_mape = mean_absolute_percentage_error(y_eval, y_eval_pred)
    test_mape = mean_absolute_percentage_error(y_test, y_test_pred)

    print('Train RMSE: %.3f, Train MAPE: %.3f' % (train_rmse, train_mape))
    print('Val RMSE: %.3f, Val MAPE: %.3f' % (eval_rmse, eval_mape))
    print('Test RMSE: %.3f, Test MAPE: %.3f' % (test_rmse, test_mape))

    error_log = {
        'model_type': 'stacking',
        'meta_learner': meta_name,
        'train_rmse': train_rmse,
        'validation_rmse': eval_rmse,
        'test_rmse': test_rmse, 
        'train_mape': train_mape,
        'validation_mape': eval_mape,
        'test_mape': test_mape
    }

    return error_log

# Collecting Error Metrics (for comparison)

In [10]:
%%time

df_error_log = []

all_metas = [linear, lasso, ridge, rfr, xgb, lgb]

for i, meta_est in enumerate(tqdm(all_metas)):
    error_log = best_meta_learner(estimators=estimators, meta_learner=meta_est, meta_name=estimators[i][0])
    
    df_error_log.append(error_log)

df_error_log = pd.DataFrame(df_error_log)
df_error_log

  0%|          | 0/6 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Train RMSE: 425.479, Train MAPE: 0.026
Val RMSE: 809.920, Val MAPE: 0.045
Test RMSE: 850.521, Test MAPE: 0.046


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Train RMSE: 440.837, Train MAPE: 0.027
Val RMSE: 809.667, Val MAPE: 0.045
Test RMSE: 852.178, Test MAPE: 0.046


...

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Train RMSE: 506.264, Train MAPE: 0.024
Val RMSE: 862.769, Val MAPE: 0.046
Test RMSE: 894.472, Test MAPE: 0.047


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Train RMSE: 472.241, Train MAPE: 0.024
Val RMSE: 836.947, Val MAPE: 0.045
Test RMSE: 861.826, Test MAPE: 0.046
CPU times: user 1min 4s, sys: 10.4 s, total: 1min 14s
Wall time: 9min 36s


,model_type,meta_learner,train_rmse,validation_rmse,test_rmse,train_mape,validation_mape,test_mape
0,stacking,linear,425.478663,809.920272,850.520971,0.025894,0.045263,0.045746
1,stacking,lasso,440.836924,809.667447,852.177580,0.026520,0.045294,0.045870
2,stacking,ridge,425.469066,809.911690,850.515217,0.025893,0.045263,0.045746
3,stacking,random_forest,492.008061,856.641215,890.283762,0.026635,0.047586,0.048020
4,stacking,xgboost,506.263661,862.769127,894.472320,0.024493,0.045906,0.046539
5,stacking,lightgbm,472.241116,836.946878,861.826255,0.024411,0.045311,0.045857


# Saving Results

In [11]:
df_error_log.to_csv('stacking_errors.csv', index=False)